In [ ]:
import numpy as np
from numpy.polynomial.polynomial import polyfit
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

%matplotlib inline

In [ ]:
df = pd.read_csv('BD 1.csv', sep=';')

In [ ]:
df.corr()['Coeficiente de atrito']

In [ ]:
df.drop(['Temperatura_ensaio', 
         'Temperatura_AISWEB', 
         'Umidade_ensaio', 
         'Operacoes_anual',
         'Operacoes_medicoes'], 
       axis=1,
       inplace=True)

In [ ]:
df.head()

In [ ]:
X = df.loc[:, ['Distancia', 'Macrotextura', 'Lado', 'Remocao', 'Idade_pavimento_meses', 'Umidade_AISWEB', 'Operacoes_remocoes']]
Y = df['Coeficiente de atrito']

In [ ]:
# Dividindo os conjuntos de dados entre 90% para Treinamento e 10% para Teste
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print('Shape X_train:', X_train.shape) # Imprime a matriz dos dados de entrada de treinamento
print('Shape X_test:', X_test.shape) # Imprime a matriz dos dados de saída de treinamento
print('Shape y_train:', y_train.shape) # Imprime a matriz dos dados de entrada de teste
print('Shape y_test:', y_test.shape)  # Imprime a matriz dos dados de saída de teste

In [ ]:
# Pré-processando os dados de entrada utilizando o pré-processamento Standard Scaler
scaler_x = StandardScaler()
scaler_x.fit(X_train) # Ajustando os dados para a média e desvio padrão do conjunto de teste
X_train = pd.DataFrame(scaler_x.transform(X_train), columns=X.columns) # Aplicando a transformação no conjunto de treinamento
X_test = pd.DataFrame(scaler_x.transform(X_test), columns=X.columns) # O conjunto de teste é pré-processado de acordo com o conjunto de treinamento

In [ ]:
# COM 1 CAMADA

ativacao = ['logistic', 'tanh', 'relu'] # funções de ativação testadas
menor_mae = 1 # o algoritmo começa a testar arquiteturas a partir de Erro Médio Absoluto de 1,00
for i in np.arange(1, 100):
    for j in ativacao:
        for k in [0.01, 0.1, 0.5]:
            reg = MLPRegressor(
                        hidden_layer_sizes=(i),
                        activation=j,
                        solver='lbfgs',
                        alpha=k,
                        random_state=0,
                        max_iter=500)
            
            reg.fit(X_train, y_train)
            mae_teste = mean_absolute_error(y_test, reg.predict(X_test))
            score_teste = reg.score(X_test, y_test)
            print('i:%i', i, 'mae_teste: %.3f' %mae_teste, 'score_teste: %.2f' %(100*score_teste), 'f ativacao: %s' %j,'alpha: %.3f' %k)
            if mae_teste < menor_mae:
                menor_mae = mae_teste
                neuronios_cam_1 = i
                score_teste_melhor = score_teste*100
                ativacao_1 = j
                alpha = k

# Salvando os resultados da Arquitetura com menor erro em um arquivo TXT                
np.savetxt('resultados_c_macro_1cam.txt',
           [menor_mae, neuronios_cam_1, 
            score_teste_melhor, 
            ativacao_1, 
            alpha],
           fmt='%.5s',
           header='menor_mae, neuronios_cam_1, score_teste_melhor, ativacao, alpha')

In [ ]:
# COM 2 CAMADAS

ativacao = ['logistic', 'tanh', 'relu'] # funções de ativação testadas
menor_mae = 1 # o algoritmo começa a testar arquiteturas a partir de Erro Médio Absoluto de 1,00
menor_mae = 1
for i in np.arange(1, 100):
    for y in np.arange(1, 100):
        for j in ativacao:
            for k in [0.01, 0.1, 0.5]:
                reg = MLPRegressor(
                            hidden_layer_sizes=(y, i),
                            activation=j,
                            solver='lbfgs',
                            alpha=k,
                            random_state=0,
                            max_iter=500)
                
                reg.fit(X_train, y_train)
                mae_teste = mean_absolute_error(y_test, reg.predict(X_test))
                score_teste = reg.score(X_test, y_test)
                print('y:', y, 'i:', i, 'mae_teste: %.3f' %mae_teste, 'score_teste: %.2f' %(100*score_teste), 'f ativacao: %s' %j,'alpha: %.3f' %k)
                if mae_teste < menor_mae:
                    menor_mae = mae_teste
                    neuronios_cam_1 = y
                    neuornios_cam_2 = i
                    score_teste_melhor = score_teste*100
                    ativacao_1 = j
                    alpha = k
        
# Salvando os resultados da Arquitetura com menor erro em um arquivo TXT        
np.savetxt('resultados_c_macro_2cam.txt',
            [menor_mae, neuronios_cam_1, 
            neuornios_cam_2, 
            score_teste_melhor, 
            ativacao_1, 
            alpha],
            fmt='%.5s',
            header='menor_mae, neuronios_cam_1, neuronios_cam_2, score_teste_melhor, ativacao, alpha')